In [1]:
import numpy as np
import torch
from torchinfo import summary
from backbones import get_model

# Init and load training model

In [2]:
model = get_model('repvgg', dropout=0.0, fp16=False, num_features=512)

In [3]:
# Load RepVGG training checkpoint
weight = torch.load('work_dirs/ms1mv3_repvgg/model.pt', map_location=torch.device('cpu'))
model.load_state_dict(weight, strict=True)
model = model.eval()

## Stats

In [4]:
x = torch.rand((1, 3, 112, 112))
with torch.no_grad():
    y_training = model(x)

In [5]:
summary(model, input_size=(1, 3, 112, 112), col_names=["num_params", "mult_adds"], depth=2)

Layer (type:depth-idx)                   Param #                   Mult-Adds
RepVGG                                   --                        --
├─RepVGGBlock: 1-1                       --                        --
│    └─Sequential: 2-1                   1,392                     4,064,352
│    └─Sequential: 2-2                   240                       451,680
│    └─Identity: 2-3                     --                        --
│    └─ReLU: 2-4                         --                        --
├─ModuleList: 1-2                        --                        --
│    └─RepVGGBlock: 2-5                  23,232                    18,063,552
│    └─RepVGGBlock: 2-6                  23,328                    18,063,648
├─ModuleList: 1-3                        --                        --
│    └─RepVGGBlock: 2-7                  46,464                    9,032,064
│    └─RepVGGBlock: 2-8                  92,736                    18,063,936
│    └─RepVGGBlock: 2-9                 

# Switch RepVGG to deployment

In [6]:
for m in model.modules():
    if hasattr(m, 'switch_to_deploy'):
        m.switch_to_deploy()

In [7]:
torch.save(model.state_dict(), 'work_dirs/ms1mv3_repvgg/model_deploy.pt')

## Stats

In [8]:
summary(model, input_size=(1, 3, 112, 112), col_names=["num_params", "mult_adds"], depth=2)

Layer (type:depth-idx)                   Param #                   Mult-Adds
RepVGG                                   --                        --
├─RepVGGBlock: 1-1                       --                        --
│    └─Conv2d: 2-1                       1,344                     4,214,784
│    └─Identity: 2-2                     --                        --
│    └─ReLU: 2-3                         --                        --
├─ModuleList: 1-2                        --                        --
│    └─RepVGGBlock: 2-4                  20,784                    16,294,656
│    └─RepVGGBlock: 2-5                  20,784                    16,294,656
├─ModuleList: 1-3                        --                        --
│    └─RepVGGBlock: 2-6                  41,568                    8,147,328
│    └─RepVGGBlock: 2-7                  83,040                    16,275,840
│    └─RepVGGBlock: 2-8                  83,040                    16,275,840
│    └─RepVGGBlock: 2-9              

In [9]:
with torch.no_grad():
    y_deploy = model(x)

# Check output

In [10]:
torch.allclose(y_training, y_deploy, rtol=1e-5, atol=1e-5)

True

In [11]:
euclidena_dist = sum(((y_deploy - y_training)**2).reshape(512))
print('Euclidean distance:', euclidena_dist.item())

Euclidean distance: 3.739324683227352e-10


In [12]:
cos = torch.nn.CosineSimilarity(dim=1)(y_training, y_deploy)
print('Cosine similarity:', cos.item())

Cosine similarity: 1.0


# MBF's stats

In [13]:
model_mbf = get_model('mbf', dropout=0.0, fp16=False, num_features=512)
model_mbf = model_mbf.eval()

In [14]:
summary(model_mbf, input_size=(1, 3, 112, 112), col_names=["num_params", "mult_adds"], depth=2)

Layer (type:depth-idx)                                       Param #                   Mult-Adds
MobileFaceNet                                                --                        --
├─ModuleList: 1-1                                            --                        --
│    └─ConvBlock: 2-1                                        3,840                     10,838,400
│    └─ConvBlock: 2-2                                        2,688                     7,225,728
│    └─DepthWise: 2-3                                        34,944                    65,129,472
│    └─Residual: 2-4                                         139,776                   106,377,216
│    └─DepthWise: 2-5                                        102,656                   38,988,800
│    └─Residual: 2-6                                         812,544                   156,862,464
│    └─DepthWise: 2-7                                        270,336                   32,342,016
│    └─Residual: 2-8                